In [1]:
import numpy
npmax = numpy.max
npmin = numpy.min
from numpy import float32, pi, imag, real, mean, maximum, nanmean, nanstd, absolute

from bokeh.plotting import Session, show, figure, output_server, hplot
from bokeh.models import Range1d
from bokeh.charts import Histogram

from gnss import codes
from gnss.receiver import channels
from gnss.signals import Signal
from gnss.receiver import sources
from gnss.filters import iir_filter
from gnss.acquisition import coarse, fine
from gnss.receiver import outputs
from gnss.tracking import tracking, CN0Algorithm
from gnss.visualization import TrackingPlot, TrackingMultiPlot, CorrelationGridPlot, \
    CorrelationLinePlot, FinePhasePlot, sample_histogram

In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
!du -hs /mnt/gluster/by-location/ascension-island/a1/gpsl2/usrp5/20130307_184310

1.3G	/mnt/gluster/by-location/ascension-island/a1/gpsl2/usrp5/20130307_184310



We are using data from the USRP front ends. This data contains *signed 4-bit complex* samples.


- L2:
    - 5MHz sampling frequency
    - 1.2276GHz center frequency

- L1:
    - 25MHz sampling
    - 1.57542GHz center frequency
    
- L5:
    - 25MHz sampling
    - 1.17645GHz center frequency

These files are large, so we should choose a reasonable buffer size for their data. 2 seconds worth of data would be:

In [4]:
5e6 * 2, 25e6 * 2

(10000000.0, 50000000.0)

which are 10 mega-samples and 50 mega-samples respectively.

In [5]:
# f_center = 1.2276e9
# file_f_samp = 5e6
# filepath = '/mnt/gluster/by-location/ascension-island/a1/gpsl2/usrp5/20130307_184310'
# source = sources.FileSource(filepath, file_f_samp=file_f_samp, f_center=f_center, buffer_size=10e6, bit_depth=4, real=False)
# source.load()
# print('we have {0} seconds of data'.format(source.buffer_size / source.f_samp))

filepath = '/mnt/gluster/by-location/ascension-island/a1/gpsl2/usrp5/20130307_184310'
source = sources.FileSource(filepath, file_f_samp=5e6, f_center=1.2276e9, buffer_size=4e6, bit_depth=4, real=False, decimation=2)

In [6]:
session = Session(root_url='http://192.168.3.137:5006/', load_from_config=False)
session.login('anon', '1234')
output_server('tracking-l2c', session=session)

In [7]:
c_acq_plot = CorrelationLinePlot()
f_acq_plot = FinePhasePlot()
show(hplot(c_acq_plot.plot, f_acq_plot.plot))

In [8]:
step = 2e-3
cn0_algo = CN0Algorithm()
f_update = 1. / step
block_duration = 2e-3
block_size = block_duration * source.f_samp - 1
output_buff_size = source.buffer_size / (step * source.f_samp)

In [9]:
i_corr_plot = TrackingPlot('i corr', output_buff_size)
q_corr_plot = TrackingPlot('q corr', output_buff_size)
phase_plot = TrackingPlot('phase error', output_buff_size)
show(hplot(i_corr_plot.plot, q_corr_plot.plot, phase_plot.plot))
buffer_size = round(source.file_duration)
doppler_plot = TrackingPlot('doppler 1Hz', buffer_size)
cn0_plot = TrackingPlot('cn0 1Hz', buffer_size)
show(hplot(doppler_plot.plot, cn0_plot.plot))

In [10]:
source.seek(time=20)
source.load()
print('we have {0} seconds of data starting at {1} seconds'.format(source.buffer_size / source.f_samp, source.buffer_start_time))

we have 1.6 seconds of data starting at 20.0 seconds


In [12]:
svid = 5
signal = Signal.GPSL2(svid)

c_acq = coarse.CoarseAcquirerLowMem(source, 20e-3, 1, dopp_min=-5000, dopp_max=5000)
f_acq = fine.FineAcquirer(source, 2e-3, 1e-3, 29)

In [13]:
c_acq.acquire(signal)
chip0 = c_acq.chip
f_dopp0 = c_acq.f_dopp
time = c_acq.time
cn0 = c_acq.cn0
print(chip0, f_dopp0, time, cn0)
c_acq_plot.update(c_acq)

1516545.5316 -2250.0 20.0 65.5481965473


In [14]:
f_acq.acquire(signal, time, chip0, f_dopp0)
f_dopp = f_acq.f_dopp
time = f_acq.time
print(chip0, f_dopp, time)
f_acq_plot.update(f_acq)

1516545.5316 -2227.75063943 20.0
0.0 0.056 -1.17124796958 -0.62011616326


In [15]:
dll_filter = iir_filter.FirstOrderLowpass(10, f_update)
pll_filter = iir_filter.FirstOrderLowpass(12, f_update)
# pll_filter = iir_filter.SecondOrderLowpass(omega_n=5 * 12, zeta=1.7, fs=f_update)
# pll_filter = iir_filter.SecondOrderLowpassV2(omega_n=5 * 32, zeta=1.7, fs=f_update)
fll_filter = iir_filter.FirstOrderLowpass(1 / pi, f_update);  fll_filter.f(0);  fll_filter.f(0)

delay = .5
correlator = tracking.Correlator(chip_delays=[-2 * delay, -delay, 0., delay, 2 * delay])

In [16]:
time = f_acq.time
theta = last_phase_error = 0.
chip = chip_out = chip0
f_dopp = f_acq.f_dopp
source.seek(time)
source.load()

In [17]:
while True:
    while time < source.max_time - block_duration:
        code_corr, dopp_corr = correlator.correlate(signal, source, block_size, time, chip, f_dopp, theta)
        earlier, early, prompt, late, later = code_corr
        chip_error = tracking.delay_discriminator(earlier, prompt, later)
        phase_error = -tracking.costas_discriminator(prompt)  # minus sign used b/c phase is subtracted in signal correlator
        chip_error = dll_filter.f(chip_error)
        phase_error = pll_filter.f(phase_error)
        dopp_error = (phase_error - last_phase_error) / (step * 2 * pi)
        last_phase_error = phase_error
        dopp_error = fll_filter.f(dopp_error)
        # error correction
        chip -= chip_error
        theta -= phase_error
#         f_dopp += dopp_error
        # carrier-aiding and propagation
        time += step
        f_chip = signal.code.rate * (1. + f_dopp / signal.f_carrier)
        chip += step * f_chip
        f_inter = signal.f_carrier - source.f_center
        theta += step * (f_inter + f_dopp) * 2. * pi
        # outputs
        chip_out = chip_out - chip_error
        i_corr_plot.push_data(time, real(prompt))
        q_corr_plot.push_data(time, imag(prompt))
        phase_plot.push_data(time, phase_error)
#         print(f_dopp)
    data = phase_plot.data.get()
    f_dopp -= mean(data) / (2 * pi) / step
    print(f_dopp)
    i_corr_plot.update()
    q_corr_plot.update()
    phase_plot.update()
    cn0 = cn0_algo.compute(i_corr_plot.data.get(), q_corr_plot.data.get(), step)
    doppler_plot.push_data(time, f_dopp)
    cn0_plot.push_data(time, cn0)
    doppler_plot.update()
    cn0_plot.update()
    source.seek(time)
    source.load()

-2231.35727492
-2237.68771711
-2243.13162913
-2245.45358273
-2245.43559625
-2247.18152519
-2246.37915532
-2246.72364074
-2246.71638902
-2246.66018563
-2247.07536774
-2246.78178483
-2246.66088205
-2246.90241901
-2246.46809821
-2247.39253364
-2247.74909169
-2246.21974692
-2246.54312834
-2246.52063155
-2247.12265526
-2246.40479583
-2246.23766263
-2246.02319195
-2246.85682107
-2246.80867781
-2246.85698547
-2246.39271839
-2246.31467161
-2247.86684089
-2245.82351857
-2247.24928058
-2247.21350384
-2247.04591582
-2246.99143099
-2245.66262952
-2246.06743029
-2246.69686252
-2246.96034968
-2246.54220269
-2246.75184758
-2247.34412553
-2247.00704313
-2247.2088681
-2247.86592251
-2247.13811431
-2247.55150442
-2247.20660903
-2246.69157744
-2247.1905038
-2248.19260004
-2246.34713071
-2246.00545505
-2247.60745945
-2247.59289467
-2247.03853264
-2247.00231576
-2246.71194546
-2246.03993415
-2246.76707039
-2246.85446012
-2247.48606177
-2246.77792113
-2247.40868505
-2247.09587979
-2247.43529916
-2248.024659

Exception: time out of file range